# GLM Experiments

This notebook contains experiments on predicting heart disease using generalized linear models.

In [73]:
import pandas as pd
import numpy as np

X_pd = pd.read_csv('229_processed_cleveland.data')
del X_pd['Unnamed: 0']
Y_pd = pd.read_csv('229_processed_cleveland_Y.data', header=None)

In [74]:
# pandas -> numpy
X = X_pd.values
Y = Y_pd.values[:,1]

In [75]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.1)

## Forward feature selection

This method allows us to pass in a model and run forward feature selection on it.

In [76]:
from sklearn import linear_model
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

def kfold_validate (X_sub, k, model):
    kf = KFold(n_splits = k)
    sum = 0
    for train, test in kf.split(X_sub):
        reg = model
        reg.fit(X_sub[train], Y_train[train])
        sum += (reg.score(X_sub[test], Y_train[test]))
    return reg, sum / k
        
def filter_select(X, Y, num_features, model):
    # ignore the bad columns
    avail_features = list(range(0, np.shape(X)[1]))
        
    included_features = [] #[17 - 1] # diabetes'
    
    best_kfold_score = -1
    best_featureset = []
    best_model_overall = None
    
    for i in range(num_features):
        best_feature = -1
        best_score = -1
        best_model = None
        
        for feature in avail_features:
            model, score = kfold_validate(X_train[:,included_features + [feature]], 5, model)
            if score > best_score:
                best_score = score
                best_feature = feature
                best_model = model
        included_features += [best_feature]
        avail_features.remove(best_feature)
        if (best_score > best_kfold_score):
            best_kfold_score = score
            best_featureset = included_features
            best_model_overall = best_model
        print(best_score)
    print('Best featureset:', list(map(lambda x: X_pd.keys()[x], best_featureset)))
    print('Test score:', best_model_overall.score(X_test[:, best_featureset], Y_test))
    return included_features

## Experiment 1: Logistic Regression on Uncertain Features

This is a straightforward logistic regression. We run it on the features that aren't all the same for healthy individuals.

In [80]:
X_full_pd = pd.read_csv('229_processed_cleveland.data')
del X_full_pd['Unnamed: 0']
Y_full_pd = pd.read_csv('229_processed_cleveland_Y_full.data', header=None)
# pandas -> numpy
X_full = X_full_pd.values
Y_full = Y_full_pd.values[:,1]

In [78]:
filter_select(X_train, Y_train, 40, linear_model.LogisticRegression())

0.762823529412
0.786588235294
0.853725490196
0.853725490196
0.857647058824
0.865568627451
0.865568627451
0.865568627451
0.865568627451
0.865568627451
0.861647058824
0.861647058824
0.857647058824
0.857725490196
0.861725490196
0.865647058824
0.865647058824
0.865647058824
0.865568627451
0.861647058824
0.857725490196
0.857725490196
0.857725490196
0.857725490196
0.853803921569
0.853803921569
0.845960784314
0.849882352941
0.845960784314
0.842039215686
0.834196078431
0.826117647059
0.842117647059
0.834117647059
0.834039215686
0.838039215686
0.834117647059
0.834117647059
0.834117647059
0.838039215686
Best featureset: ['thal3', 'ca', 'exang', 'cigs', 'tpeakbps', 'id', 'prop', 'xhypo', 'restecg1', 'thal-9', 'dm', 'dm_invalid', 'thal6', 'lvx3', 'met', 'cmo', 'ekgmo', 'slope3', 'fbs', 'years', 'lvf', 'htn', 'ekgyr', 'cyr', 'thaldur', 'cp1', 'cday', 'lvx4', 'famhist', 'tpeakbpd', 'chol', 'thaltime', 'restecg2', 'oldpeak', 'cp3', 'cp2', 'slope1', 'restecg0', 'nitr', 'thal7']
Test score: 0.8620689655

[50,
 32,
 28,
 6,
 24,
 0,
 15,
 29,
 45,
 52,
 9,
 53,
 49,
 36,
 21,
 33,
 11,
 46,
 8,
 7,
 38,
 4,
 13,
 35,
 19,
 39,
 34,
 37,
 10,
 25,
 5,
 20,
 43,
 30,
 41,
 42,
 48,
 44,
 16,
 51]

## Experiment 2: Original features
Extracting the original features and testing them.

In [117]:
orig_feats = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
feat_indices = []
for i in range(len(X_pd.keys())):
    k = X_pd.keys()[i]
    for o in orig_feats:
        if k == o or (k.startswith(o) and k[len(k) - 1].isnumeric()):
            feat_indices.append(i)
print("Number of predictors:", len(feat_indices))

Number of predictors: 23


In [118]:
def filter_select_orig(X, Y, num_features, model):
    # ignore the bad columns
    avail_features = feat_indices[:]
        
    included_features = [] #[17 - 1] # diabetes'
    
    best_kfold_score = -1
    best_featureset = []
    best_model_overall = None
    
    for i in range(num_features):
        best_feature = -1
        best_score = -1
        best_model = None
        
        for feature in avail_features:
            model, score = kfold_validate(X_train[:,included_features + [feature]], 5, model)
            if score > best_score:
                best_score = score
                best_feature = feature
                best_model = model
        included_features += [best_feature]
        avail_features.remove(best_feature)
        if (best_score > best_kfold_score):
            best_kfold_score = score
            best_featureset = included_features
            best_model_overall = best_model
        print(best_score)
    print('Best featureset:', list(map(lambda x: X_pd.keys()[x], best_featureset)))
    print('Test score:', best_model_overall.score(X_test[:, best_featureset], Y_test))
    return included_features

In [119]:
filter_select_orig(X_train, Y_train, len(feat_indices), linear_model.LogisticRegression())

0.762823529412
0.786588235294
0.853725490196
0.853725490196
0.853725490196
0.853725490196
0.853647058824
0.857568627451
0.853647058824
0.845803921569
0.841803921569
0.837960784314
0.830039215686
0.822117647059
0.817960784314
0.845568627451
0.845490196078
0.845411764706
0.841411764706
0.845411764706
0.837411764706
0.837647058824
0.837647058824
Best featureset: ['thal3', 'ca', 'exang', 'restecg1', 'thal6', 'cp2', 'thalach', 'slope3', 'thal-9', 'cp1', 'thal7', 'chol', 'age', 'fbs', 'cp4', 'oldpeak', 'sex', 'cp3', 'slope2', 'slope1', 'trestbps', 'restecg0', 'restecg2']
Test score: 0.758620689655


[50,
 32,
 28,
 45,
 49,
 42,
 22,
 46,
 52,
 39,
 51,
 5,
 1,
 8,
 40,
 30,
 2,
 41,
 47,
 48,
 3,
 44,
 43]

In [120]:
from sklearn import svm

filter_select_orig(X_train, Y_train, len(feat_indices), svm.LinearSVC())

0.762823529412
0.786588235294
0.818196078431
0.829960784314
0.837960784314
0.841882352941
0.845803921569
0.849725490196
0.849725490196
0.845803921569
0.837960784314
0.845882352941
0.849568627451
0.865411764706
0.865411764706
0.865411764706
0.865411764706
0.857490196078
0.853490196078
0.793490196078
0.829333333333
0.774509803922
0.63937254902
Best featureset: ['thal3', 'ca', 'cp4', 'restecg2', 'exang', 'fbs', 'slope3', 'restecg0', 'restecg1', 'cp2', 'sex', 'thal6', 'slope1', 'oldpeak', 'slope2', 'thal7', 'thal-9', 'cp1', 'cp3', 'thalach', 'trestbps', 'age', 'chol']
Test score: 0.275862068966


[50,
 32,
 40,
 43,
 28,
 8,
 46,
 44,
 45,
 42,
 2,
 49,
 48,
 30,
 47,
 51,
 52,
 39,
 41,
 22,
 3,
 1,
 5]

In [ ]:
from sklearn import svm

filter_select(X_train, Y_train, 40, svm.LinearSVC())

0.762823529412
0.778588235294
0.818196078431
0.837803921569
0.849647058824
0.853725490196
0.857647058824
0.861647058824
0.865647058824
0.865647058824
0.865647058824
0.865647058824
0.865647058824
0.861725490196
0.849803921569
0.849803921569
0.849803921569
0.849647058824
0.857647058824
0.861568627451
0.861568627451
0.865725490196
0.869568627451
0.865568627451


In [112]:
feat_indices

[]

In [5]:
import scipy.stats as stats

diabetic = X[:, 17 - 1] == 1
stats.f_oneway(Y[diabetic], Y[np.logical_not(diabetic)])

TypeError: unhashable type: 'slice'

Exploratory data analysis using ANOVA:

ANOVA indicates a p-value of 0.09, a little promising. Now let's try basic logistic regression:

In [146]:
np.mean(X[Y==1, 50]),np.mean(X[Y==0, 50])

(5.6879999999999997, 3.7006369426751591)

In [45]:
Y

0      0
1      1
2      1
3      0
4      0
5      0
6      1
7      0
8      1
9      1
10     0
11     0
12     1
13     0
14     0
15     0
16     1
17     0
18     0
19     0
20     0
21     0
22     1
23     1
24     1
25     0
26     0
27     0
28     0
29     1
      ..
252    0
253    0
254    0
255    0
256    0
257    0
258    0
259    1
260    0
261    1
262    0
263    0
264    1
265    1
266    1
267    1
268    1
269    0
270    1
271    0
272    1
273    0
274    1
275    0
276    0
277    0
278    1
279    0
280    1
281    0
Name: 1, Length: 282, dtype: int64

In [58]:
X

array([   0.,    1.,    2.,    3.,    4.,    5.,    6.,    7.,    8.,
          9.,   10.,   11.,   12.,   13.,   14.,   15.,   16.,   17.,
         18.,   19.,   20.,   21.,   22.,   23.,   24.,   25.,   26.,
         27.,   28.,   29.,   30.,   31.,   32.,   33.,   34.,   35.,
         36.,   37.,   38.,   39.,   40.,   41.,   42.,   43.,   44.,
         45.,   46.,   47.,   48.,   49.,   50.,   51.,   52.,   53.,
         54.,   55.,   56.,   57.,   58.,   59.,   60.,   61.,   62.,
         63.,   64.,   65.,   66.,   67.,   68.,   69.,   70.,   71.,
         72.,   73.,   74.,   75.,   76.,   77.,   78.,   79.,   80.,
         81.,   82.,   83.,   84.,   85.,   86.,   87.,   88.,   89.,
         90.,   91.,   92.,   93.,   94.,   95.,   96.,   97.,   98.,
         99.,  100.,  101.,  102.,  103.,  104.,  105.,  106.,  107.,
        108.,  109.,  110.,  111.,  112.,  113.,  114.,  115.,  116.,
        117.,  118.,  119.,  120.,  121.,  122.,  123.,  124.,  125.,
        126.,  127.,

In [64]:
X_pd.keys()

Index(['Unnamed: 0', 'id', 'age', 'sex', 'trestbps', 'htn', 'chol', 'cigs',
       'years', 'fbs', 'dm', 'famhist', 'ekgmo', 'ekgday', 'ekgyr', 'dig',
       'prop', 'nitr', 'pro', 'diuretic', 'thaldur', 'thaltime', 'met',
       'thalach', 'thalrest', 'tpeakbps', 'tpeakbpd', 'dummy', 'trestbpd',
       'exang', 'xhypo', 'oldpeak', 'rldv5e', 'ca', 'thal', 'cmo', 'cday',
       'cyr', 'lvx3', 'lvx4', 'lvf', 'cp1', 'cp4', 'cp3', 'cp2', 'restecg2',
       'restecg0', 'restecg1', 'slope3', 'slope2', 'slope1', 'dm_invalid'],
      dtype='object')

In [35]:
X_pd.keys()[36]

'lvx3'